# TP04 — Differential Kinematics (Session 2: Exercises 5-8)

**GEII3 — Industrie 4.0: Robotique | Lab Session 4 - Part 2**

Advanced topics: Manipulability ellipsoid, velocity control, force analysis, applications.

---

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Ellipse
from math import cos, sin, pi, sqrt
from scipy.linalg import svd, det, pinv, eig

try:
    import roboticstoolbox as rtb
    from spatialmath import SE3
    RTB_AVAILABLE = True
except ImportError:
    RTB_AVAILABLE = False

np.set_printoptions(precision=4, suppress=True)
print("✓ Session 2 ready")

---

# Exercise 5: Manipulability Analysis & Ellipsoid

**Estimated time**: 60 minutes

## Task 5.1: Manipulability Measures (20 min)

### Yoshikawa's Manipulability

$$w = \sqrt{\det(J J^T)}$$

- $w = 0$: singular configuration
- $w > 0$: non-singular (higher is better)
- Maximum when robot is "far from singularities"

### Condition Number

$$\kappa = \frac{\sigma_{\max}}{\sigma_{\min}}$$

- $\kappa = 1$: isotropic (equal dexterity in all directions)
- $\kappa \to \infty$: near singularity

### Minimum Singular Value

$$\sigma_{\min} = \min(\sigma_i)$$

- Direct measure of "distance" to singularity
- Used for singularity avoidance

---

## Task 5.2: Implementation & Comparison (20 min)

In [ ]:
L1, L2 = 0.4, 0.3
dh_2r = [(0,0,L1,0,'r'), (0,0,L2,0,'r')]

print("\n" + "="*70)
print("  MANIPULABILITY MEASURES COMPARISON")
print("="*70)

test_configs = [
    (np.array([pi/4, pi/2]), "Good config"),
    (np.array([0, 0]), "Extended (singular)"),
    (np.array([pi/2, pi]), "Folded (singular)"),
    (np.array([pi/4, -pi/4]), "Moderate"),
]

results = []

print(f"\n{'Configuration':<20} {'Yoshikawa':<12} {'κ (cond.)':<12} {'σ_min':<12} {'Status'}")
print("-" * 75)

for q, desc in test_configs:
    J = jacobian_2r_geometric(q, L1, L2)
    
    # Yoshikawa
    w = abs(det(J))
    
    # SVD analysis
    U, s, Vt = svd(J)
    kappa = s[0] / s[-1] if s[-1] > 1e-10 else np.inf
    sigma_min = s[-1]
    
    # Status
    if w < 0.001:
        status = "SINGULAR"
    elif w < 0.05:
        status = "Near singular"
    else:
        status = "Good"
    
    print(f"{desc:<20} {w:<12.6f} {kappa:<12.2f} {sigma_min:<12.6f} {status}")
    
    results.append((desc, q, w, kappa, sigma_min))

print("\n** Observations:**")
print("  - Yoshikawa = 0 → Singular")
print("  - κ → ∞ → Near singularity")
print("  - σ_min → 0 → Approaching singularity")

## Task 5.3: Manipulability Ellipsoid (20 min)

The **manipulability ellipsoid** visualizes the robot's velocity capabilities.

For unit joint velocities, end-effector velocities form an ellipsoid:
$$\mathbf{v}^T (J J^T)^{-1} \mathbf{v} = 1$$

Ellipsoid axes are given by SVD: $J = U \Sigma V^T$
- Principal axes: columns of $U$
- Axis lengths: singular values in $\Sigma$

In [ ]:
def plot_manipulability_ellipse_2d(J, ax, center=(0,0), scale=1.0, label=""):
    """
    Plot 2D manipulability ellipse from 2×2 Jacobian.
    """
    # SVD
    U, s, Vt = svd(J)
    
    # Ellipse parameters
    width = 2 * s[0] * scale
    height = 2 * s[1] * scale
    angle = np.arctan2(U[1,0], U[0,0]) * 180 / pi
    
    ellipse = Ellipse(center, width, height, angle=angle,
                      fill=False, edgecolor='blue', linewidth=2, label=label)
    ax.add_patch(ellipse)
    
    # Draw principal axes
    for i in range(2):
        axis = U[:, i] * s[i] * scale
        ax.arrow(center[0], center[1], axis[0], axis[1],
                 head_width=0.02, head_length=0.01, fc='red', ec='red')
    
    return ellipse


# Visualize ellipsoids for different configurations
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

print("\n" + "="*70)
print("  MANIPULABILITY ELLIPSOID VISUALIZATION")
print("="*70)

for idx, (desc, q, w, kappa, sigma_min) in enumerate(results[:4]):
    J = jacobian_2r_geometric(q, L1, L2)
    
    # Robot configuration
    _, T_list = fk_dh(dh_2r, q)
    x = [T[0,3] for T in T_list]
    y = [T[1,3] for T in T_list]
    
    ax = axes[idx]
    ax.plot(x, y, 'o-', linewidth=2, markersize=8, color='gray', label='Robot')
    ax.plot(x[0], y[0], 'gs', markersize=10)
    ax.plot(x[-1], y[-1], 'r*', markersize=12)
    
    # Ellipse at end-effector
    plot_manipulability_ellipse_2d(J, ax, center=(x[-1], y[-1]), 
                                    scale=0.5, label='Velocity ellipse')
    
    ax.set_xlim(-0.2, 0.8)
    ax.set_ylim(-0.2, 0.8)
    ax.axis('equal')
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_title(f"{desc}\nw={w:.4f}, κ={kappa:.1f}", fontweight='bold')
    ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

print("\n** Interpretation:**")
print("  - Large ellipse → High manipulability")
print("  - Thin ellipse → Near singularity (one direction hard)")
print("  - Circular ellipse → Isotropic (κ ≈ 1)")

---

# Exercise 6: Velocity Kinematics & Resolved Motion Control

**Estimated time**: 70 minutes

## Task 6.1: Forward Velocity Problem (15 min)

Given joint velocities, find end-effector velocity: $\mathbf{v} = J \dot{\mathbf{q}}$

In [ ]:
print("\n" + "="*70)
print("  FORWARD VELOCITY KINEMATICS")
print("="*70)

# Configuration
q = np.array([pi/4, pi/3])
q_dot = np.array([0.5, -0.3])  # rad/s

print(f"\nConfiguration: q = {np.rad2deg(q).astype(int)}°")
print(f"Joint velocities: q_dot = {q_dot} rad/s")

# Jacobian
J = jacobian_2r_geometric(q, L1, L2)
print(f"\nJacobian:")
print(J)

# End-effector velocity
v = forward_velocity(J, q_dot)
print(f"\nEnd-effector velocity:")
print(f"  v_x = {v[0]:.4f} m/s")
print(f"  v_y = {v[1]:.4f} m/s")
print(f"  |v| = {np.linalg.norm(v):.4f} m/s")

## Task 6.2: Inverse Velocity Problem (20 min)

Given desired end-effector velocity, find joint velocities: $\dot{\mathbf{q}} = J^{-1} \mathbf{v}_d$

In [ ]:
print("\n" + "="*70)
print("  INVERSE VELOCITY KINEMATICS")
print("="*70)

# Desired Cartesian velocity
v_desired = np.array([0.1, 0.05])  # m/s in X and Y

print(f"\nDesired end-effector velocity: {v_desired} m/s")
print(f"Configuration: q = {np.rad2deg(q).astype(int)}°")

# Method 1: Direct inverse (if square and non-singular)
det_J = det(J)
if abs(det_J) > 1e-6:
    J_inv = np.linalg.inv(J)
    q_dot_direct = J_inv @ v_desired
    print(f"\nMethod 1: Direct inverse")
    print(f"  q_dot = {q_dot_direct} rad/s")
else:
    print(f"\nSingular! Cannot use direct inverse")

# Method 2: Pseudo-inverse (always works)
q_dot_pinv = inverse_velocity(J, v_desired, method='pinv')
print(f"\nMethod 2: Pseudo-inverse")
print(f"  q_dot = {q_dot_pinv} rad/s")

# Method 3: Damped Least Squares
q_dot_dls = inverse_velocity(J, v_desired, method='dls', damping=0.05)
print(f"\nMethod 3: Damped LS (λ=0.05)")
print(f"  q_dot = {q_dot_dls} rad/s")

# Verify
v_achieved_direct = J @ q_dot_direct if abs(det_J) > 1e-6 else None
v_achieved_pinv = J @ q_dot_pinv
v_achieved_dls = J @ q_dot_dls

print(f"\n--- Verification ---")
print(f"Desired:      {v_desired}")
if v_achieved_direct is not None:
    print(f"Direct:       {v_achieved_direct} (error: {np.linalg.norm(v_desired - v_achieved_direct):.2e})")
print(f"Pseudo-inv:   {v_achieved_pinv} (error: {np.linalg.norm(v_desired - v_achieved_pinv):.2e})")
print(f"Damped LS:    {v_achieved_dls} (error: {np.linalg.norm(v_desired - v_achieved_dls):.2e})")

## Task 6.3: Resolved Motion Rate Control (35 min)

Track a Cartesian trajectory using velocity control.

**Algorithm**:
```
while not at target:
    1. Compute position error: e = p_target - p_current
    2. Desired velocity: v_d = K * e (proportional control)
    3. Compute Jacobian: J = J(q)
    4. Joint velocities: q_dot = J^{-1} * v_d
    5. Integrate: q = q + q_dot * dt
```

In [ ]:
def resolved_motion_control(dh_params, q_init, target, K=1.0, dt=0.01, 
                            max_iter=500, tol=0.001):
    """
    Resolved Motion Rate Control for trajectory tracking.
    
    Parameters:
    - dh_params: robot DH table
    - q_init: initial configuration
    - target: desired end-effector position [x, y]
    - K: proportional gain
    - dt: time step
    - max_iter: maximum iterations
    - tol: convergence tolerance
    
    Returns:
    - q_traj: joint trajectory
    - p_traj: Cartesian trajectory
    - converged: success flag
    """
    q = q_init.copy()
    q_traj = [q.copy()]
    p_traj = []
    
    for iteration in range(max_iter):
        # Current position
        T, _ = fk_dh(dh_params, q)
        p_current = extract_position(T)[:2]  # XY only for 2D
        p_traj.append(p_current)
        
        # Error
        error = target - p_current
        error_norm = np.linalg.norm(error)
        
        # Check convergence
        if error_norm < tol:
            return np.array(q_traj), np.array(p_traj), True
        
        # Desired velocity (proportional control)
        v_desired = K * error
        
        # Jacobian
        J = jacobian_2r_geometric(q, L1, L2)
        
        # Joint velocities
        q_dot = pinv(J) @ v_desired
        
        # Integrate (Euler)
        q = q + q_dot * dt
        q_traj.append(q.copy())
    
    return np.array(q_traj), np.array(p_traj), False


# Test RMRC
print("\n" + "="*70)
print("  RESOLVED MOTION RATE CONTROL")
print("="*70)

q_start = np.array([pi/6, pi/4])
target_pos = np.array([0.4, 0.4])

print(f"\nStarting config: q = {np.rad2deg(q_start).astype(int)}°")
print(f"Target position: {target_pos}")

q_traj, p_traj, converged = resolved_motion_control(
    dh_2r, q_start, target_pos, K=2.0, dt=0.02
)

print(f"\nConverged: {converged}")
print(f"Iterations: {len(q_traj)}")
print(f"Final position: {p_traj[-1]}")
print(f"Final error: {np.linalg.norm(target_pos - p_traj[-1]):.4f} m")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Cartesian path
axes[0].plot(p_traj[:, 0], p_traj[:, 1], 'b-', linewidth=2, label='Actual path')
axes[0].plot(p_traj[0, 0], p_traj[0, 1], 'go', markersize=12, label='Start')
axes[0].plot(target_pos[0], target_pos[1], 'r*', markersize=15, label='Target')
axes[0].set_xlabel('X (m)', fontsize=12)
axes[0].set_ylabel('Y (m)', fontsize=12)
axes[0].set_title('Cartesian Trajectory', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axis('equal')
axes[0].legend()

# Joint trajectories
time = np.arange(len(q_traj)) * 0.02
axes[1].plot(time, np.rad2deg(q_traj[:, 0]), 'b-', linewidth=2, label='q1')
axes[1].plot(time, np.rad2deg(q_traj[:, 1]), 'r--', linewidth=2, label='q2')
axes[1].set_xlabel('Time (s)', fontsize=12)
axes[1].set_ylabel('Joint Angle (deg)', fontsize=12)
axes[1].set_title('Joint Trajectories', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.show()

---

# Exercise 7: Force Analysis & Statics

**Estimated time**: 50 minutes

## Task 7.1: Force-Torque Relationship (25 min)

### Principle of Virtual Work

Power balance:
$$\boldsymbol{\tau}^T \dot{\mathbf{q}} = \mathbf{F}^T \mathbf{v}$$

Substituting $\mathbf{v} = J \dot{\mathbf{q}}$:
$$\boldsymbol{\tau}^T \dot{\mathbf{q}} = \mathbf{F}^T J \dot{\mathbf{q}}$$

Therefore:
$$\boldsymbol{\tau} = J^T \mathbf{F}$$

This is the **force-torque duality**.

In [ ]:
print("\n" + "="*70)
print("  FORCE-TORQUE MAPPING")
print("="*70)

# Configuration
q = np.array([pi/3, pi/4])
J = jacobian_2r_geometric(q, L1, L2)

print(f"\nConfiguration: q = {np.rad2deg(q).astype(int)}°")
print(f"\nJacobian:")
print(J)

# Scenario 1: Given force, find torques
F_ee = np.array([10.0, 5.0])  # N in X and Y

tau = force_to_torque(J, F_ee)

print(f"\n[Scenario 1] Force → Torque")
print(f"  End-effector force: {F_ee} N")
print(f"  Required joint torques: {tau} Nm")

# Scenario 2: Given torques, find force
tau_given = np.array([2.0, 1.5])  # Nm

F_from_tau = torque_to_force(J, tau_given)

print(f"\n[Scenario 2] Torque → Force")
print(f"  Joint torques: {tau_given} Nm")
print(f"  Resulting end-effector force: {F_from_tau} N")

# Verify reciprocity
tau_verify = force_to_torque(J, F_from_tau)
print(f"\n  Verification (should match): {tau_verify} Nm")
print(f"  Error: {np.linalg.norm(tau_given - tau_verify):.2e}")

## Task 7.2: Static Equilibrium (25 min)

Find torques to hold a payload against gravity.

In [ ]:
print("\n" + "="*70)
print("  STATIC EQUILIBRIUM WITH PAYLOAD")
print("="*70)

# Payload
m_payload = 2.0  # kg
g = 9.81  # m/s^2
F_gravity = np.array([0, -m_payload * g])  # Force in -Y direction

print(f"\nPayload: {m_payload} kg")
print(f"Gravity force: {F_gravity} N")

# Test different configurations
configs = [
    (np.array([0, pi/2]), "Vertical"),
    (np.array([pi/4, pi/4]), "45° extended"),
    (np.array([pi/2, 0]), "Horizontal"),
]

print(f"\n{'Configuration':<20} {'τ1 (Nm)':<12} {'τ2 (Nm)':<12} {'|τ| (Nm)'}")
print("-" * 60)

for q, desc in configs:
    J = jacobian_2r_geometric(q, L1, L2)
    tau = force_to_torque(J, F_gravity)
    tau_norm = np.linalg.norm(tau)
    
    print(f"{desc:<20} {tau[0]:>10.4f}  {tau[1]:>10.4f}  {tau_norm:>10.4f}")

print("\n** Observation: Torque requirements vary with configuration!")
print("   Some configs require much higher torques for same payload.")

---

# Exercise 8: Advanced Topics & Applications

**Estimated time**: 60 minutes

## Task 8.1: Singularity Avoidance (30 min)

Modify velocity control to avoid singularities.

In [ ]:
def resolved_motion_singularity_robust(dh_params, q_init, target, 
                                       K=1.0, dt=0.01, lambda_max=0.1,
                                       max_iter=500, tol=0.001):
    """
    RMRC with Damped Least Squares for singularity robustness.
    Damping increases near singularities.
    """
    q = q_init.copy()
    q_traj = [q.copy()]
    p_traj = []
    manip_traj = []
    
    for iteration in range(max_iter):
        T, _ = fk_dh(dh_params, q)
        p_current = extract_position(T)[:2]
        p_traj.append(p_current)
        
        error = target - p_current
        if np.linalg.norm(error) < tol:
            return np.array(q_traj), np.array(p_traj), np.array(manip_traj), True
        
        v_desired = K * error
        
        J = jacobian_2r_geometric(q, L1, L2)
        w = abs(det(J))  # Manipulability
        manip_traj.append(w)
        
        # Adaptive damping
        if w < 0.01:  # Near singularity
            lambda_damp = lambda_max
        else:
            lambda_damp = lambda_max * (1 - w / 0.1) if w < 0.1 else 0.001
        
        # Damped Least Squares
        JJT = J @ J.T
        J_inv_dls = J.T @ np.linalg.inv(JJT + lambda_damp**2 * np.eye(2))
        q_dot = J_inv_dls @ v_desired
        
        q = q + q_dot * dt
        q_traj.append(q.copy())
    
    return np.array(q_traj), np.array(p_traj), np.array(manip_traj), False


print("\n" + "="*70)
print("  SINGULARITY-ROBUST CONTROL")
print("="*70)

# Start near singularity, move through it
q_start_sing = np.array([0, np.deg2rad(10)])  # Nearly extended
target_sing = np.array([0.3, 0.5])

print(f"\nStarting near singularity: q = {np.rad2deg(q_start_sing).astype(int)}°")
print(f"Target: {target_sing}")

# Standard RMRC (may fail)
print("\nMethod 1: Standard RMRC (pseudo-inverse)...")
q_traj_std, p_traj_std, conv_std = resolved_motion_control(
    dh_2r, q_start_sing, target_sing, K=1.5, dt=0.02
)
print(f"  Converged: {conv_std}, Iterations: {len(q_traj_std)}")

# Robust RMRC
print("\nMethod 2: Singularity-robust (DLS)...")
q_traj_rob, p_traj_rob, manip_rob, conv_rob = resolved_motion_singularity_robust(
    dh_2r, q_start_sing, target_sing, K=1.5, dt=0.02, lambda_max=0.1
)
print(f"  Converged: {conv_rob}, Iterations: {len(q_traj_rob)}")

# Compare
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Paths
axes[0].plot(p_traj_std[:, 0], p_traj_std[:, 1], 'b-', 
             linewidth=2, label='Standard RMRC')
axes[0].plot(p_traj_rob[:, 0], p_traj_rob[:, 1], 'r--', 
             linewidth=2, label='Robust RMRC')
axes[0].plot(target_sing[0], target_sing[1], 'g*', markersize=15, label='Target')
axes[0].set_xlabel('X (m)')
axes[0].set_ylabel('Y (m)')
axes[0].set_title('Cartesian Paths', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].axis('equal')

# Joint trajectories (robust)
time_rob = np.arange(len(q_traj_rob)) * 0.02
axes[1].plot(time_rob, np.rad2deg(q_traj_rob[:, 0]), 'b-', linewidth=2, label='q1')
axes[1].plot(time_rob, np.rad2deg(q_traj_rob[:, 1]), 'r--', linewidth=2, label='q2')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Joint Angle (deg)')
axes[1].set_title('Joint Trajectories (Robust)', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Manipulability evolution
axes[2].plot(time_rob[:len(manip_rob)], manip_rob, 'g-', linewidth=2)
axes[2].axhline(y=0.01, color='r', linestyle='--', label='Singularity threshold')
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('Manipulability')
axes[2].set_title('Manipulability Evolution', fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Task 8.2: Summary & Key Takeaways (30 min)

### Implementation Checklist

✅ **Jacobian computation**: 3 methods (analytical, geometric, numerical)  
✅ **Singularity detection**: manipulability, condition number, SVD  
✅ **Velocity kinematics**: forward/inverse, multiple methods  
✅ **Force analysis**: torque mapping, statics  
✅ **Practical control**: RMRC, singularity avoidance  

### Industrial Applications

1. **Trajectory Following**: Use RMRC for smooth Cartesian paths
2. **Force Control**: Apply controlled forces (polishing, assembly)
3. **Singularity Avoidance**: Critical for reliability
4. **Workspace Optimization**: Position robot for best manipulability
5. **Load Estimation**: Predict torques for different payloads

### Common Pitfalls

⚠ **Near singularities**: Always use damped methods  
⚠ **Frame conventions**: Base frame vs tool frame  
⚠ **Units**: Radians for angles, consistent length units  
⚠ **Integration**: Small timesteps for velocity control  
⚠ **Joint limits**: Add constraints in real implementations  

---

In [ ]:
import pandas as pd

# Summary table
summary = {
    'Method': [
        'Analytical Jacobian',
        'Geometric Jacobian',
        'Numerical Jacobian',
        'Pseudo-inverse',
        'Damped LS',
        'RMRC',
    ],
    'Best For': [
        'Simple robots, verification',
        'Any robot, intuitive',
        'Complex robots, quick impl.',
        'Redundant robots',
        'Near singularities',
        'Cartesian control',
    ],
    'Complexity': [
        'High (derivation)',
        'Medium',
        'Low',
        'Low',
        'Low',
        'Medium',
    ],
    'Speed': [
        'Fast',
        'Medium',
        'Slow',
        'Fast',
        'Fast',
        'Real-time',
    ],
    'Robustness': [
        'Exact',
        'Exact',
        'Approx.',
        'Poor at sing.',
        'Excellent',
        'Good',
    ]
}

df = pd.DataFrame(summary)

print("\n" + "="*80)
print("  JACOBIAN METHODS - SUMMARY TABLE")
print("="*80)
print("\n" + df.to_string(index=False))

print("\n\n" + "="*80)
print("  RECOMMENDATIONS FOR GEII ENGINEERS")
print("="*80)
print("""
1. LEARNING / EDUCATION:
   → Start with analytical (understand the math)
   → Practice geometric (builds intuition)
   → Use numerical for verification

2. INDUSTRIAL IMPLEMENTATION:
   → Geometric Jacobian (most versatile)
   → Always check manipulability
   → Use Damped LS near singularities
   → Implement velocity limits and joint limits

3. TRAJECTORY CONTROL:
   → RMRC for Cartesian paths
   → Monitor manipulability online
   → Adaptive damping based on configuration
   → Combine with joint-space planning

4. FORCE CONTROL:
   → Use J^T for force mapping
   → Consider payload in all configurations
   → Account for link masses (dynamics)
   → Validate torque limits

5. DEBUGGING TIPS:
   → Visualize manipulability ellipsoid
   → Compare multiple Jacobian methods
   → Validate with RTB
   → Test at known singular configs
   → Check determinant and singular values
""")

print("\n" + "="*80)
print("  LAB COMPLETE - CONGRATULATIONS! 🎓")
print("="*80)
print("""
You have mastered:
✓ Three methods to compute Jacobians
✓ Singularity analysis and detection
✓ Velocity kinematics (forward and inverse)
✓ Force-torque relationships
✓ Practical control algorithms (RMRC)
✓ Robustness techniques for real robots

Next steps:
→ TP05: Trajectory Planning (smooth motion generation)
→ TP06: Robot Dynamics (forces, inertia, gravity)
→ TP07: Control (PID, computed torque, impedance)
""")